In [11]:
from telegram import Update
from telegram.ext import filters, MessageHandler, ApplicationBuilder, ContextTypes, CommandHandler

In [22]:
API_TOKEN = 'MY_TOKEN_HERE' # Token hidden for security reasons
application = ApplicationBuilder().token(API_TOKEN).build()

In [23]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id, text="I'm your personal AI assistant, please talk to me!")
    
start_handler = CommandHandler('start', start)
application.add_handler(start_handler)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model = AutoModelForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")

In [24]:
def generate_response(question, model, tokenizer):
    prompt = f"Use only your past knowledge about animals and answer the following question concisely and accurately:\n\n{question}\n"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs.input_ids, 
        num_return_sequences=1,
        do_sample=True,
        temperature=0.7, 
        top_k=50, 
        top_p=0.95
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)[len(prompt):]


In [20]:
animal_name = "lion"
question = f"Give me facts about {animal_name}"
result = generate_response(question, model, tokenizer)

In [21]:
print(result)


1. Name: lion

2. Classification: mammal

3. Order: Carnivora

4. Family: Felidae

5. Genus: Pan

6. Species: Panthera leo

7. Size: Adult males can reach up to 6 feet in length and weigh up to 500 pounds, while females can be up to 5 feet in length and weigh up to 250 pounds.

8. Habitat: The lion inhabits various regions of sub-Saharan Africa, including Kenya, Tanzania, Uganda, and South Africa.

9. Diet: The lion is a top predator, feeding on large prey such as antelopes, zebras, giraffes, and impalas.

10. Behavior: Lions are solitary animals, except during the breeding season when they form harems with a mate and one or more offspring.

11. Ecology: The lion's habitat is the savannah, where they hunt and hunt in groups called prides.

12. Conservation: The lion population has been declining due to poaching, habitat loss, and competition with humans for resources.

13. Interesting Facts: The lion is the second-largest cat after the tiger. Its paw prints are 3.6 inches in length an

In [25]:
async def llm_response(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await context.bot.send_message(chat_id=update.effective_chat.id, text=f"Your query is: \"{update.message.text}\"\nPlease wait, while I generate a response...")
    response = generate_response(update.message.text, model, tokenizer)
    await context.bot.send_message(chat_id=update.effective_chat.id, text=response)

llm_handler = MessageHandler(filters.TEXT & (~filters.COMMAND), llm_response)
application.add_handler(llm_handler)

In [26]:
await application.initialize()
await application.start()

In [27]:
await application.updater.start_polling(poll_interval=5.0, drop_pending_updates=True)

<Queue at 0x1e18d0d1fc0 maxsize=0 _getters[1]>

In [28]:
await application.updater.stop()

In [29]:
await application.stop()
await application.shutdown()